In [139]:
from bs4 import BeautifulSoup
import os
import requests
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from urllib.request import urlopen
from urllib import request
from urllib.error import HTTPError
from urllib.error import URLError
from selenium.common.exceptions import UnexpectedAlertPresentException

merchants = pd.read_excel('fuzz.xlsx')
w = 'wiki'
c = 'crunch'

options = Options()
options.add_argument("--headless")
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override","Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0")
driver = webdriver.Firefox(profile, options=options)

In [140]:
### GET_TICKER VERSION 1


# def getParentWebpage(html):
#     try:
#         trs = html.find('table', {'class': 'infobox vcard'}).find_all('tr')
#         for tr in trs:
#             if 'Parent' in tr.text:
#                 path = str(tr.find('td').find('a').get('href'))
#                 driver.get('https://en.wikipedia.org' + path)
#                 html = BeautifulSoup(driver.page_source, "lxml")
#                 return getParentWebpage(html)
#         return html
#     except:
#         return html

# def getTickerOnline(merchant):
#     try:
#         driver.get('https://google.com/search?q=' + str(merchant) + ' wikipedia')
#         html = BeautifulSoup(driver.page_source, "lxml")
#         for res in html.find_all('div', {'class': 'g'}):
#             if 'wikipedia' in res.find('a').get('href'):
#                 driver.get(res.find('a').get('href'))
#                 html = BeautifulSoup(driver.page_source, "lxml")
#                 break
#         trs = html.find('table', {'class': 'infobox vcard'}).find_all('tr')        
#         for tr in trs:
#             if 'traded' in tr.text.lower():
#                 return tr.find('a', {'rel':'nofollow'}).text
#         html = getParentWebpage(html)
#         for tr in trs:    
#             if 'private' in tr.text.lower():
#                 return 'PRIVATE'
#         return 'ERR'
#     except:
#         return np.nan

In [141]:
### GET_TICKER VERSION 2 (WIKIPEDIA)


def getTickerW(merchant):
    html = getHTMLW(merchant)
    if isTradedW(html) != False:
        return isTradedW(html)
    if isPrivateW(html) == True:
        return 'PRIVATE'
    else:
        return 'ERR'

def getHTMLW(merchant):
    try:
        driver.get('https://google.com/search?q=' + str(merchant) + ' wiki')
        html = BeautifulSoup(driver.page_source, "lxml")
        for res in html.find_all('div', {'class': 'g'}):
            if 'wikipedia' in res.find('a').get('href'):
                driver.get(res.find('a').get('href'))
                return BeautifulSoup(driver.page_source, "lxml")
        return 'ERR'
    except:
        return 'ERR'
    
def findBoxW(html, ret):
    try:
        return html.find('table', {'class': 'infobox vcard'}).find_all('tr')
    except:
        try:
            return html.find('table', {'class': 'infobox vevent'}).find_all('tr')
        except:
            try:
                return html.find('table', {'class': 'infobox hproduct'}).find_all('tr')
            except:
                try:
                    return html.find('table', {'class': 'infobox'}).find_all('tr')
                except:
                    return ret

def isTradedW(html):
    try:
        if type(html) == str:
            return False
        trs = findBoxW(html, False)
        for tr in trs:
            try:
                txt = tr.find('th').text.lower()
            except:
                continue
            if 'traded' in txt:
                try:
                    return tr.find('a', {'rel':'nofollow'}).text
                except:
                    try:
                        return tr.find('td').text.replace(':', ' ').replace('-', ' ').split()[1]
                    except:
                        continue
        return isTradedW(getParentW(html))
    except:
        return False

def isPrivateW(html):
    try:
        if type(html) == str:
            return False
        trs = findBoxW(html, False) 
        for tr in trs:
            if 'private' in tr.text.lower():
                return True
        return isPrivateW(getParentW(html))
    except:
        return False

def getParentW(html):
    try:
        trs = findBoxW(html, 'ERR')
        for tr in trs:
            try:
                txt = tr.find('th').text.lower()
            except:
                continue
            if 'parent' in txt:
                path = tr.find('td').find_all('a')[-1].get('href')
                if '#' in path:
                    path = tr.find('td').find_all('a')[-2].get('href')
                driver.get('https://en.wikipedia.org' + path)
                return BeautifulSoup(driver.page_source, "lxml")
        for tr in trs:
            try:
                txt = tr.find('th').text.lower()
            except:
                continue
            if 'own' in txt or 'developer' in txt or 'successor' in txt:
                path = tr.find('td').find_all('a')[-1].get('href')
                if '#' in path:
                    path = tr.find('td').find_all('a')[-2].get('href')
                driver.get('https://en.wikipedia.org' + path)
                html = BeautifulSoup(driver.page_source, "lxml")
                trs = html.find('table', {'class': 'infobox vcard'}).find_all('tr')
                for tr in trs:
                    if 'born' in tr.text.lower():
                        break
                    else:
                        return html
        for tr in trs:
            if 'subsidiary of' in tr.text.lower():
                try:
                    path = tr.find('td').find_all('a')[-1].get('href')
                    if '#' in path:
                        path = tr.find('td').find_all('a')[-2].get('href')
                    driver.get('https://en.wikipedia.org' + path)
                    return BeautifulSoup(driver.page_source, "lxml")
                except:
                    return getHTMLW(tr.find('td').text.split('of')[1])
        return 'ERR'
    except:
        return 'ERR'

In [142]:
### GET_TICKER VERSION 2 EXTENSION (CRUNCHBASE)


def getTickerC(merchant):
    html = getHTMLC(merchant)
    if isTradedC(html) != False:
        return isTradedC(html)
    if isPrivateC(html):
        return 'PRIVATE'
    else:
        return 'ERR'
    
def getHTMLC(merchant):
    try:
        driver.get('https://google.com/search?q=' + str(merchant) + ' crunchbase')
        html = BeautifulSoup(driver.page_source, "lxml")
        for res in html.find_all('div', {'class': 'g'}):
            if 'crunchbase' in res.find('a').get('href'):
                driver.get(res.find('a').get('href'))
                return BeautifulSoup(driver.page_source, "lxml")
        return 'ERR'
    except:
        return 'ERR'

def isTradedC(html):
    try:
        if type(html) == str:
            return False
        if 'public' in html.find_all('fields-card')[1].text.split()[2].lower():
            return  html.find_all('fields-card')[1].text.split()[4].split(':')[1]
        else:
            return isTradedC(getParentC(html))
    except:
        return False
    
def isPrivateC(html):
    try:
        if type(html) == str:
            return False
        if 'private' in html.find_all('fields-card')[1].text.split()[2].lower() or 'delisted' in html.find_all('fields-card')[1].text.split()[2].lower():
            return True
        else:
            return isPrivate(getParent(html))
    except:
        return False

def getParentC(html):
    try:
        spans = html.find_all('fields-card')[0].find_all('span')
        for i in range(len(spans)):
            try:
                if 'Sub-Organization' in spans[i].text:
                    driver.get('https://www.crunchbase.com' + spans[i+5].find('a').get('href'))
                    return BeautifulSoup(driver.page_source, "lxml")
            except:
                continue
        spans = html.find('big-values-card').find_all('span')
        for i in range(len(spans)):
            try:
                if 'Acquired' in spans[i].text:
                    driver.get('https://www.crunchbase.com' + spans[i+6].find('a').get('href'))
                    return BeautifulSoup(driver.page_source, "lxml")
            except:
                continue
        else:
            return 'ERR'
    except:
        return 'ERR'

def recentAC(merchant):
    try:
        html = getHTMLC(merchant)
        sections = html.find_all('entity-section')
        for section in sections:
            if 'Acquisitions' in section.find('div').text:
                sub_html = section
                break
        trs = sub_html.find('list-card').find('tbody').find_all('tr')
        acq = []
        date = []
        price = []
        for tr in trs:
            acq.append(tr.find_all('td')[0].text)
            date.append(tr.find_all('td')[1].text)
            price.append(tr.find_all('td')[2].text)
        data = {'Acquisition': acq, 'Date': date, 'Price': price}
        return pd.DataFrame(data=data)
    except:
        return 'ERR'

In [143]:
### COMBINING VERSIONS - FINAL VERSION
###ENTER w or c for first (for 'Wikipedia' or 'Crunchbase')

def getTicker(merchant, first):
    if first == 'crunch':
        tkr = getTickerC(merchant)
        if tkr != 'ERR':
            return tkr
        else:
            return getTickerW(merchant)
    if first == 'wiki':
        tkr = getTickerW(merchant)
        if tkr != 'ERR':
            return tkr
        else:
            return getTickerC(merchant)

In [144]:
### COMPARING THE TWO BOTS

In [145]:
companies = pd.read_excel('Completedomains1010_Manual.xlsx')

In [146]:
cs = companies['companyName']

In [147]:
wiki = []
#crunch = []
for c in cs:
    wiki.append(getTickerW(c))
    time.sleep(2.2)
    #crunch.append(getTickerC(c))
    #time.sleep(2.2)

In [148]:
data = {'company': cs, 'wiki': wiki, 'crunch': crunch}

In [149]:
df = pd.DataFrame(data=data)

In [150]:
df['correct'] = companies['ticker_manual']

In [151]:
df.dropna(inplace=True)

In [152]:
def fun(corr, w, c):
    if corr == w and corr == c:
        return 'both'
    if corr == w:
        return 'wiki'
    if corr == c:
        return 'crunch'
    else:
        return np.nan

In [153]:
df['who_correct'] = df.apply(lambda x: fun(x['correct'], x['wiki'], x['crunch']), axis=1)

In [160]:
df['wiki'].value_counts().head(2)

PRIVATE    49
ERR        33
Name: wiki, dtype: int64

In [161]:
df['crunch'].value_counts().head(2)

PRIVATE    75
ERR         5
Name: crunch, dtype: int64

In [162]:
df['who_correct'].value_counts()

both      157
wiki       36
crunch     21
Name: who_correct, dtype: int64

In [169]:
df.to_excel('PerformanceReport_getTicker.xlsx')